In [1]:
from pyspark.sql import SparkSession

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
0,application_1613506460428_0001,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [2]:
spark = SparkSession\
        .builder\
        .appName('franciscoM')\
        .master('Local[*]')\
        .enableHiveSupport()\
        .getOrCreate()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [3]:
spark

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

## Carga de datos

In [12]:
df_spark = spark.read.csv("s3://focobucket/titanic/train.csv", header = True)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [13]:
df_spark.show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+
|PassengerId|Survived|Pclass|                Name|   Sex| Age|SibSp|Parch|          Ticket|   Fare|Cabin|Embarked|
+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+
|          1|       0|     3|Braund, Mr. Owen ...|  male|  22|    1|    0|       A/5 21171|   7.25| null|       S|
|          2|       1|     1|Cumings, Mrs. Joh...|female|  38|    1|    0|        PC 17599|71.2833|  C85|       C|
|          3|       1|     3|Heikkinen, Miss. ...|female|  26|    0|    0|STON/O2. 3101282|  7.925| null|       S|
|          4|       1|     1|Futrelle, Mrs. Ja...|female|  35|    1|    0|          113803|   53.1| C123|       S|
|          5|       0|     3|Allen, Mr. Willia...|  male|  35|    0|    0|          373450|   8.05| null|       S|
|          6|       0|     3|    Moran, Mr. James|  male|null|    0|    0|      

In [17]:
df_spark.printSchema()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- PassengerId: string (nullable = true)
 |-- Survived: string (nullable = true)
 |-- Pclass: string (nullable = true)
 |-- Name: string (nullable = true)
 |-- Sex: string (nullable = true)
 |-- Age: string (nullable = true)
 |-- SibSp: string (nullable = true)
 |-- Parch: string (nullable = true)
 |-- Ticket: string (nullable = true)
 |-- Fare: string (nullable = true)
 |-- Cabin: string (nullable = true)
 |-- Embarked: string (nullable = true)

In [19]:
from pyspark.sql.types import DoubleType

df_spark = df_spark.withColumn("Survived", df_spark["Survived"].cast(DoubleType()))
df_spark = df_spark.withColumn("Pclass", df_spark["Pclass"].cast(DoubleType()))
df_spark = df_spark.withColumn("Age", df_spark["Age"].cast(DoubleType()))
df_spark = df_spark.withColumn("SibSp", df_spark["SibSp"].cast(DoubleType()))
df_spark = df_spark.withColumn("Parch", df_spark["Parch"].cast(DoubleType()))
df_spark = df_spark.withColumn("Fare", df_spark["Fare"].cast(DoubleType()))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

## Registros Nulos

In [20]:
Dict_Null = {col:df_spark.filter(df_spark[col].isNull()).count()/df_spark.count() for col in df_spark.columns}
Dict_Null

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

{'PassengerId': 0.0, 'Survived': 0.0, 'Pclass': 0.0, 'Name': 0.0, 'Sex': 0.0, 'Age': 0.19865319865319866, 'SibSp': 0.0, 'Parch': 0.0, 'Ticket': 0.0, 'Fare': 0.0, 'Cabin': 0.7710437710437711, 'Embarked': 0.002244668911335578}

Debido a que Cabin, es una columna coon muchos valores nulos, sera eliminada del dataset. Por otro lado, la columna Age, los valores nulos seran remplazados por la mediana de la columna. Para Finalizar, los valores nulos de la columna Embarked seran eliminado del dataset.

In [22]:
from pyspark.ml.feature import Imputer
from pyspark.sql.functions import col, when, mean
from pyspark.sql.functions import udf

#Age
mediana = df_spark.approxQuantile("age", [0.5], 0)[0]
df_spark = df_spark.fillna( { 'age':mediana } )

#Cabin
df_spark = df_spark.drop('Cabin')

#Embarked
df_spark = df_spark.na.drop(how='any')
df_spark.show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+--------+
|PassengerId|Survived|Pclass|                Name|   Sex| Age|SibSp|Parch|          Ticket|   Fare|Embarked|
+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+--------+
|          1|     0.0|   3.0|Braund, Mr. Owen ...|  male|22.0|  1.0|  0.0|       A/5 21171|   7.25|       S|
|          2|     1.0|   1.0|Cumings, Mrs. Joh...|female|38.0|  1.0|  0.0|        PC 17599|71.2833|       C|
|          3|     1.0|   3.0|Heikkinen, Miss. ...|female|26.0|  0.0|  0.0|STON/O2. 3101282|  7.925|       S|
|          4|     1.0|   1.0|Futrelle, Mrs. Ja...|female|35.0|  1.0|  0.0|          113803|   53.1|       S|
|          5|     0.0|   3.0|Allen, Mr. Willia...|  male|35.0|  0.0|  0.0|          373450|   8.05|       S|
|          6|     0.0|   3.0|    Moran, Mr. James|  male|28.0|  0.0|  0.0|          330877| 8.4583|       Q|
|          7|     0

In [23]:
Dict_Null = {col:df_spark.filter(df_spark[col].isNull()).count()/df_spark.count() for col in df_spark.columns}
Dict_Null

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

{'PassengerId': 0.0, 'Survived': 0.0, 'Pclass': 0.0, 'Name': 0.0, 'Sex': 0.0, 'Age': 0.0, 'SibSp': 0.0, 'Parch': 0.0, 'Ticket': 0.0, 'Fare': 0.0, 'Embarked': 0.0}

## Analisis Vector Objetivo

In [24]:
df_spark.groupBy('Survived').count().show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------+-----+
|Survived|count|
+--------+-----+
|     1.0|  340|
|     0.0|  549|
+--------+-----+

El dataset esta un poco desbalanceado a las personas que murieron, pero no demasiado, por lo que no es muy necesario hacer metodos de oversampling de muestra. No obstante, el modelo va a predecir mejor a las personas que murieron a que no murieron, ya que tienen mas datos para la prediccion

## Columas : Ticket, Name y PassenderID

In [25]:
df_spark.groupBy('Ticket').count().sort('count', ascending = False).show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------------+-----+
|      Ticket|count|
+------------+-----+
|    CA. 2343|    7|
|      347082|    7|
|        1601|    7|
|     CA 2144|    6|
|      347088|    6|
|     3101295|    6|
|      382652|    5|
|S.O.C. 14879|    5|
|       17421|    4|
|        4133|    4|
|      113781|    4|
|      349909|    4|
|        LINE|    4|
|       19950|    4|
|        2666|    4|
|      347077|    4|
|    PC 17757|    4|
|  W./C. 6608|    4|
|      113760|    4|
|       35273|    3|
+------------+-----+
only showing top 20 rows

In [26]:
df_spark.groupBy('Name').count().sort('count', ascending = False).show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+-----+
|                Name|count|
+--------------------+-----+
|"Duff Gordon, Lad...|    1|
|Rush, Mr. Alfred ...|    1|
|Bystrom, Mrs. (Ka...|    1|
|     Foo, Mr. Choong|    1|
|Lobb, Mr. William...|    1|
|Alexander, Mr. Wi...|    1|
|"Mullens, Miss. K...|    1|
| Ringhini, Mr. Sante|    1|
|       Ling, Mr. Lee|    1|
| Moor, Master. Meier|    1|
|Chambers, Mrs. No...|    1|
|Barton, Mr. David...|    1|
|Sandstrom, Mrs. H...|    1|
|    Osman, Mrs. Mara|    1|
|White, Mr. Richar...|    1|
|Hoyt, Mr. William...|    1|
|Silverthorne, Mr....|    1|
|Serepeca, Miss. A...|    1|
|Roebling, Mr. Was...|    1|
|   Karaic, Mr. Milan|    1|
+--------------------+-----+
only showing top 20 rows

La columna name y ticket representan el nombre y el ticket de la persona del titanic, se puede hacer distintos analisis mas detallado para estas columnas, pero por mayor simplificadad, no las voy a contablizar en el modelo, eliminandolas del dataset.

In [27]:
#Name, ticket y Passenger
df_spark = df_spark.drop('Name')
df_spark = df_spark.drop('Ticket')
df_spark = df_spark.drop('PassengerId')
df_spark.show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------+------+------+----+-----+-----+-------+--------+
|Survived|Pclass|   Sex| Age|SibSp|Parch|   Fare|Embarked|
+--------+------+------+----+-----+-----+-------+--------+
|     0.0|   3.0|  male|22.0|  1.0|  0.0|   7.25|       S|
|     1.0|   1.0|female|38.0|  1.0|  0.0|71.2833|       C|
|     1.0|   3.0|female|26.0|  0.0|  0.0|  7.925|       S|
|     1.0|   1.0|female|35.0|  1.0|  0.0|   53.1|       S|
|     0.0|   3.0|  male|35.0|  0.0|  0.0|   8.05|       S|
|     0.0|   3.0|  male|28.0|  0.0|  0.0| 8.4583|       Q|
|     0.0|   1.0|  male|54.0|  0.0|  0.0|51.8625|       S|
|     0.0|   3.0|  male| 2.0|  3.0|  1.0| 21.075|       S|
|     1.0|   3.0|female|27.0|  0.0|  2.0|11.1333|       S|
|     1.0|   2.0|female|14.0|  1.0|  0.0|30.0708|       C|
|     1.0|   3.0|female| 4.0|  1.0|  1.0|   16.7|       S|
|     1.0|   1.0|female|58.0|  0.0|  0.0|  26.55|       S|
|     0.0|   3.0|  male|20.0|  0.0|  0.0|   8.05|       S|
|     0.0|   3.0|  male|39.0|  1.0|  5.0| 31.275|       

## Analisis Multi-Variado

**Pclass**

In [28]:
df_spark.groupBy('Survived','Pclass').count().show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------+------+-----+
|Survived|Pclass|count|
+--------+------+-----+
|     1.0|   1.0|  134|
|     0.0|   1.0|   80|
|     1.0|   3.0|  119|
|     1.0|   2.0|   87|
|     0.0|   2.0|   97|
|     0.0|   3.0|  372|
+--------+------+-----+

Como se puede apreciar, la clase 3 es la clase mas propensa a morir, y la clase 1 contiene mas sobreviviente. Esto quiere decir, que en el titanic, la clase de la persona, si fue un factor importante en la sobrevivincia de las personas. En la pelicula uno veia, que las personas con mayores recuersos, entraron a los botes primero.

**Sexo**

In [29]:
df_spark.groupBy('Survived','Sex').count().sort('Survived').show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------+------+-----+
|Survived|   Sex|count|
+--------+------+-----+
|     0.0|female|   81|
|     0.0|  male|  468|
|     1.0|female|  231|
|     1.0|  male|  109|
+--------+------+-----+

En la pelicula, tambien se veia que los niños y las mujeres fueron entradas primeros a los botes, lo que, se conside con los resultados encontrados. Las mujeres eran mas propensas a sobrevivir que los hombres

**Age**

In [30]:
print('Edad Promedio de los sobreviviente:')
print(df_spark.filter(df_spark.Survived == 1).select(mean('age')).show())

print('Edad Promedio de los NO sobreviviente:')
print(df_spark.filter(df_spark.Survived == 0).select(mean('age')).show())

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Edad Promedio de los sobreviviente:
+------------------+
|          avg(age)|
+------------------+
|28.163735294117647|
+------------------+

None
Edad Promedio de los NO sobreviviente:
+------------------+
|          avg(age)|
+------------------+
|30.028233151183972|
+------------------+

None

Se puede apreciar que no hay una gran diferencia entre el promedio de las edad de los sobreviviente con los no sobreviviente, por lo que, la edad podria seruna variable no estan relevante a la hora del modelo.

**Familia Tamaña**

In [31]:
df_spark = df_spark.withColumn('tamaño_familiar',col('Parch') + col('SibSp') + 1  )
df_spark.show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------+------+------+----+-----+-----+-------+--------+---------------+
|Survived|Pclass|   Sex| Age|SibSp|Parch|   Fare|Embarked|tamaño_familiar|
+--------+------+------+----+-----+-----+-------+--------+---------------+
|     0.0|   3.0|  male|22.0|  1.0|  0.0|   7.25|       S|            2.0|
|     1.0|   1.0|female|38.0|  1.0|  0.0|71.2833|       C|            2.0|
|     1.0|   3.0|female|26.0|  0.0|  0.0|  7.925|       S|            1.0|
|     1.0|   1.0|female|35.0|  1.0|  0.0|   53.1|       S|            2.0|
|     0.0|   3.0|  male|35.0|  0.0|  0.0|   8.05|       S|            1.0|
|     0.0|   3.0|  male|28.0|  0.0|  0.0| 8.4583|       Q|            1.0|
|     0.0|   1.0|  male|54.0|  0.0|  0.0|51.8625|       S|            1.0|
|     0.0|   3.0|  male| 2.0|  3.0|  1.0| 21.075|       S|            5.0|
|     1.0|   3.0|female|27.0|  0.0|  2.0|11.1333|       S|            3.0|
|     1.0|   2.0|female|14.0|  1.0|  0.0|30.0708|       C|            2.0|
|     1.0|   3.0|female| 

**Fare**

In [32]:
print('Precio Promedio de los sobreviviente:')
print(df_spark.filter(df_spark.Survived == 1).select(mean('Fare')).show())

print('Precio Promedio de los NO sobreviviente:')
print(df_spark.filter(df_spark.Survived == 0).select(mean('Fare')).show())

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Precio Promedio de los sobreviviente:
+------------------+
|         avg(Fare)|
+------------------+
|48.209498235294106|
+------------------+

None
Precio Promedio de los NO sobreviviente:
+------------------+
|         avg(Fare)|
+------------------+
|22.117886885245877|
+------------------+

None

Del precio promedio, se puede desprender que las personas con mayores precio del ticket, tenian mayor propencion a sobrevivir, mas que las personas con ticket bajos

**Embarked**

In [33]:
df_spark.groupBy('Survived','Embarked').count().sort('Survived').show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------+--------+-----+
|Survived|Embarked|count|
+--------+--------+-----+
|     0.0|       S|  427|
|     0.0|       C|   75|
|     0.0|       Q|   47|
|     1.0|       C|   93|
|     1.0|       Q|   30|
|     1.0|       S|  217|
+--------+--------+-----+

## Transformacion y Reproceso de columnas

In [34]:
# Transformamos a vectores OneHot las columnas "importantes"
from pyspark.ml.feature import StringIndexer

important_columns = ['Sex', 'Embarked']
indexed_important_columns = []

for col in important_columns:
  indexer = StringIndexer(inputCol=col, outputCol=col + 'Index')
  indexed_important_columns.append(col + 'Index')
  df_spark = indexer.fit(df_spark).transform(df_spark)

df_spark.show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------+------+------+----+-----+-----+-------+--------+---------------+--------+-------------+
|Survived|Pclass|   Sex| Age|SibSp|Parch|   Fare|Embarked|tamaño_familiar|SexIndex|EmbarkedIndex|
+--------+------+------+----+-----+-----+-------+--------+---------------+--------+-------------+
|     0.0|   3.0|  male|22.0|  1.0|  0.0|   7.25|       S|            2.0|     0.0|          0.0|
|     1.0|   1.0|female|38.0|  1.0|  0.0|71.2833|       C|            2.0|     1.0|          1.0|
|     1.0|   3.0|female|26.0|  0.0|  0.0|  7.925|       S|            1.0|     1.0|          0.0|
|     1.0|   1.0|female|35.0|  1.0|  0.0|   53.1|       S|            2.0|     1.0|          0.0|
|     0.0|   3.0|  male|35.0|  0.0|  0.0|   8.05|       S|            1.0|     0.0|          0.0|
|     0.0|   3.0|  male|28.0|  0.0|  0.0| 8.4583|       Q|            1.0|     0.0|          2.0|
|     0.0|   1.0|  male|54.0|  0.0|  0.0|51.8625|       S|            1.0|     0.0|          0.0|
|     0.0|   3.0|  m

In [36]:
# Transformamos los indices en vectores
from pyspark.ml.feature import OneHotEncoderEstimator

vectorized_important_columns = [a.replace('Index','Vec') for a in indexed_important_columns]

encoder = OneHotEncoderEstimator(inputCols=indexed_important_columns,
                                 outputCols=vectorized_important_columns)
modelVec = encoder.fit(df_spark)
encoded = modelVec.transform(df_spark)
encoded.show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------+------+------+----+-----+-----+-------+--------+---------------+--------+-------------+-------------+-------------+
|Survived|Pclass|   Sex| Age|SibSp|Parch|   Fare|Embarked|tamaño_familiar|SexIndex|EmbarkedIndex|       SexVec|  EmbarkedVec|
+--------+------+------+----+-----+-----+-------+--------+---------------+--------+-------------+-------------+-------------+
|     0.0|   3.0|  male|22.0|  1.0|  0.0|   7.25|       S|            2.0|     0.0|          0.0|(1,[0],[1.0])|(2,[0],[1.0])|
|     1.0|   1.0|female|38.0|  1.0|  0.0|71.2833|       C|            2.0|     1.0|          1.0|    (1,[],[])|(2,[1],[1.0])|
|     1.0|   3.0|female|26.0|  0.0|  0.0|  7.925|       S|            1.0|     1.0|          0.0|    (1,[],[])|(2,[0],[1.0])|
|     1.0|   1.0|female|35.0|  1.0|  0.0|   53.1|       S|            2.0|     1.0|          0.0|    (1,[],[])|(2,[0],[1.0])|
|     0.0|   3.0|  male|35.0|  0.0|  0.0|   8.05|       S|            1.0|     0.0|          0.0|(1,[0],[1.0])|(2,[0],

## VectorAssempler

In [37]:
from pyspark.ml.feature import VectorAssembler


inputCols = [
 'Pclass',
 'Age',
 'tamaño_familiar',
 'Fare',
 'SexVec',
 'EmbarkedVec'
]

outputCol = "feactures"
df_vector = VectorAssembler(inputCols = inputCols, outputCol = outputCol)


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

## Split train y test

In [41]:
train_vector, test_vector = encoded.randomSplit([0.7,0.3])

train_vector = df_vector.transform(train_vector)
test_vector = df_vector.transform(test_vector)

train_vector.select('feactures').show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+
|           feactures|
+--------------------+
|[1.0,2.0,4.0,151....|
|[1.0,25.0,4.0,151...|
|[1.0,50.0,1.0,28....|
|[1.0,19.0,6.0,263...|
|[1.0,21.0,2.0,77....|
|[1.0,24.0,2.0,247...|
|[1.0,27.0,3.0,211...|
|[1.0,28.0,1.0,0.0...|
|[1.0,28.0,1.0,27....|
|[1.0,28.0,1.0,30....|
|[1.0,28.0,1.0,31....|
|[1.0,28.0,1.0,35....|
|[1.0,28.0,1.0,39....|
|[1.0,28.0,1.0,50....|
|[1.0,28.0,1.0,221...|
|[1.0,29.0,1.0,30....|
|[1.0,29.0,2.0,66....|
|[1.0,30.0,1.0,27....|
|[1.0,31.0,1.0,50....|
|[1.0,31.0,2.0,52....|
+--------------------+
only showing top 20 rows

## Modelo

In [42]:
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.classification import GBTClassifier
from pyspark.ml.classification import DecisionTreeClassifier

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [43]:
lr = LogisticRegression(featuresCol = 'feactures', labelCol = 'Survived', maxIter=10)
lrModel = lr.fit(train_vector)

gbt = GBTClassifier(labelCol="Survived",
                    featuresCol="feactures")

gbt_model = gbt.fit(train_vector)

dt = DecisionTreeClassifier(labelCol="Survived", featuresCol="feactures")
tres_modelo = dt.fit(train_vector)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [62]:
from pyspark.mllib.evaluation import BinaryClassificationMetrics, MulticlassMetrics

def metricas(modelo, df, tipomodelo):
    predictionAndTarget = modelo.transform(df).select("Survived", "prediction")
    metrics_binary = BinaryClassificationMetrics(predictionAndTarget.rdd.map(tuple))
    metrics_multi = MulticlassMetrics(predictionAndTarget.rdd.map(tuple))
    
    acc = metrics_multi.accuracy
    f1 = metrics_multi.fMeasure(1.0)
    precision = metrics_multi.precision(1.0)
    recall = metrics_multi.recall(1.0)
    auc = metrics_binary.areaUnderROC
    
    print('Los resultados del modelo {}'.format(tipomodelo))
    print('\n')
    print("La Accuracy del modelo es {}:".format(acc))
    print("El F1 del modelo es: {}".format(f1))
    print("La precision del modelo es: {}".format(precision))
    print("La recall del modelo es: {}".format(recall))
    print("La curva AUC del modelo es: {}".format(auc))  
    print('\n')

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [63]:
modelos_lista = [
    {
        "modelo": lrModel,
        "nombre": "RegresionLogistica"
    },
    {
        "modelo": gbt_model,
        "nombre": "GTBClassifier"
    },
    {
        "modelo": tres_modelo,
        "nombre": "Arbol de Decision"
    }
]

for model in modelos_lista:
    metricas(model["modelo"], test_vector, model["nombre"])

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Los resultados del modelo RegresionLogistica


La Accuracy del modelo es 0.7789855072463768:
El F1 del modelo es: 0.680628272251309
La precision del modelo es: 0.6632653061224489
La recall del modelo es: 0.6989247311827957
La curva AUC del modelo es: 0.7592984311651684


Los resultados del modelo GTBClassifier


La Accuracy del modelo es 0.8152173913043478:
El F1 del modelo es: 0.7357512953367875
La precision del modelo es: 0.7244897959183674
La recall del modelo es: 0.7473684210526316
La curva AUC del modelo es: 0.7990985751672


Los resultados del modelo Arbol de Decision


La Accuracy del modelo es 0.8007246376811594:
El F1 del modelo es: 0.7342995169082126
La precision del modelo es: 0.7755102040816326
La recall del modelo es: 0.6972477064220184
La curva AUC del modelo es: 0.7827555897379552

El mejor modelo es el GTBClassifier, con un curva AUC del 0.79908, y una accuracy del 0.815 para el test_vector